In [16]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
import pandas as pd
import numpy as np

In [173]:
# List of files in each month of social distancing
# TODO: Change folder and folder_fips according to drive

folder = '/content/drive/My Drive/Grad Courses/CSE547/COVID19 data analysis/safegraph/social-distancing/2020/'
folder_fips = "/content/drive/My Drive/Grad Courses/CSE547/COVID19 data analysis/safegraph/safegraph_open_census_data/metadata/cbg_fips_codes.csv"

#Change month accordingly
month = '01'

# read file names in given month
file_list = [(x[0]+ '/' + str(*x[2])) for x in os.walk(folder+month+'/')]
file_list = file_list[1:]
file_list[0]

'/content/drive/My Drive/Grad Courses/CSE547/COVID19 data analysis/safegraph/social-distancing/2020/01/01/2020-01-01-social-distancing.csv'

In [0]:
# Combine state, county with fips file and remove unrequired columns

def clean(table):
  l = []
  for val in table.origin_census_block_group:
    arr = [int(i) for i in str(val)]
    if len(arr)<12:
      arr.insert(0,0)
    state = arr[0]*10+arr[1]
    s_c = state *1000 + arr[2]*100+arr[3]*10+arr[4]
    l.append(s_c)

  # Add state,county identifier extracted from cbg
  table['unique']= l
  table['date_range_start'] =table['date_range_start'].apply(lambda x: x.split('T')[0])
  table = table[['date_range_start','device_count','completely_home_device_count','unique']]
  table = pd.merge(table,fips[['state','county','unique']],on='unique', how='inner')
  
  # data = df[['date_range_start','device_count','completely_home_device_count','unique','state','county']]
  table = table.groupby(['state', 'county','date_range_start']).mean().reset_index()

  return table

In [0]:
# Read CSV file

fips = pd.read_csv(folder_fips)
fips['unique']= fips.state_fips*1000+fips.county_fips
print(fips.head())

# Read each day csv, clean data, add to month list
month_l = []
for files in file_list:
  print(files)
  table = pd.read_csv(files)
  table = clean(table)
  month_l.append(table)

# Convert to dataframe
month_1 = pd.concat(month_l,ignore_index=True)

# Sort based on date
month_1.sort_values(by=['date_range_start'],ignore_index=True, inplace=True)

# save month file
# month_1.to_feather(folder+month+'_county_level_device_home')


In [170]:
# Verify
month_2 = pd.read_feather(folder+month+'_county_level_device_home')
month_2

,state,county,date_range_start,device_count,completely_home_device_count,unique
0,WY,Weston County,2020-04-01,70.800000,27.000000,56045.0
1,KY,Montgomery County,2020-04-01,103.222222,33.722222,21173.0
2,KY,Monroe County,2020-04-01,68.000000,19.300000,21171.0
3,KY,Metcalfe County,2020-04-01,60.500000,19.600000,21169.0
4,KY,Mercer County,2020-04-01,79.117647,28.764706,21167.0
...,...,...,...,...,...,...
77334,KY,Metcalfe County,2020-04-24,54.700000,11.100000,21169.0
77335,KY,Monroe County,2020-04-24,63.700000,13.300000,21171.0
77336,KY,Montgomery County,2020-04-24,89.666667,26.055556,21173.0
77337,KY,Martin County,2020-04-24,38.400000,11.700000,21159.0
